# Objectives
- Summarize how interactive plots can be useful to Decision Makers
- Differentiate between exploratory data visualization and data visualization to illustrates analysis results
- Use Dash to create interactive plots

# Static Interactive Plots

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

### Bubble Chart

In [ ]:
df = px.data.gapminder().query("year==2007")

fig = px.scatter(df, x="gdpPercap", y="lifeExp", size="pop", color="continent",
                 hover_name="country", log_x=True, size_max=60)
fig.show()

### Geo Scatter Plot

In [ ]:
fig = px.scatter_geo(df, locations="iso_alpha",
                     size="pop", # size of markers, "pop" is one of the columns of gapminder
                     color="continent")
fig.show()

# Dynamic Interactive Plots
Those interactive plots above are pretty sweet!! But, can we make it even better? **OF COURSE!!** We can make an animation to help tell our story!

In [ ]:
df = px.data.gapminder()
px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

In [ ]:
px.scatter_geo(df, locations='iso_alpha', size='pop', animation_frame='year', animation_group='country',
               color='continent', hover_name='country')

In [ ]:
px.bar(df, x="continent", y="pop", color="continent", 
       animation_frame="year", animation_group="country", range_y=[0,4000000000])

In [ ]:
px.choropleth(df, locations="iso_alpha", color="lifeExp", hover_name="country", animation_frame="year", range_color=[20,80])

These are just a few of the interactive plots I thought could be useful to tell our COVID story from the case study. Check out all of the options for interactive plots [here](https://plotly.com/python/)!!

# Dash
- Take some to underatand [what Dash is](https://medium.com/plotly/introducing-dash-5ecf7191b503) and what it offers for analysts to tell a story about data. 
- Once you have a decent understanding of how Dash works, check out some of [these cool examples](https://dash-gallery.plotly.host/Portal/)!

In [ ]:
import dash
import dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.iris()
all_dims = ['sepal_length', 'sepal_width', 
            'petal_length', 'petal_width']

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} 
                 for x in all_dims],
        value=all_dims[:2],
        multi=True
    ),
    dcc.Graph(id="splom"),
])

@app.callback(
    Output("splom", "figure"), 
    [Input("dropdown", "value")])
def update_bar_chart(dims):
    fig = px.scatter_matrix(
        df, dimensions=dims, color="species")
    return fig

app.run_server(debug=True, use_reloader=False)

In [ ]:
token = 'pk.eyJ1IjoiYzE0a2V2aW5jYXJkZW5hcyIsImEiOiJja25qM21wZnYwMjIyMnFqeHZxdzd5ejlnIn0.gUF8otyl_q68MCHcIvcv1Q'

df = px.data.election()
geojson = px.data.election_geojson()
candidates = df.winner.unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("Candidate:"),
    dcc.RadioItems(
        id='candidate', 
        options=[{'value': x, 'label': x} 
                 for x in candidates],
        value=candidates[0],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="choropleth"),
])

@app.callback(
    Output("choropleth", "figure"), 
    [Input("candidate", "value")])
def display_choropleth(candidate):
    fig = px.choropleth_mapbox(
        df, geojson=geojson, color=candidate,
        locations="district", featureidkey="properties.district",
        center={"lat": 45.5517, "lon": -73.7073}, zoom=9,
        range_color=[0, 6500])
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        mapbox_accesstoken=token)

    return fig

app.run_server(debug=True, use_reloader=False)